# Data Extraction from Private Property.ng Website


In [1]:
# Importing Libraries

import requests
from bs4 import BeautifulSoup 
import re

# importing the library to scrap image
import os
from urllib.parse import urljoin

In [2]:
# Error handling function while loading the site and
# Defining the Header for using User-Agent approach to have access to the site info

URL ="https://privateproperty.ng/property-for-rent"

HEADER = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"
}

def make_connection(url):
    try:
        response = requests.get(url, headers=HEADER)
        if response.status_code == 200:
            soup_content = BeautifulSoup(response.content, 'html.parser')
            return soup_content
        else:
            print(f"Resources Not available! Status Code {response.status_code}")
    except Exception as e:
        print(f"An Error occurs. Message: {e}")

In [3]:
content = make_connection(url = URL)

In [4]:
## Property for Rent

In [5]:
property_info = content.find('div', class_="similar-listings-item")

In [6]:
# property title
property_info.find("div", class_= "similar-listings-info").find('h2').get_text()

'Fully Furnished 3 Bedroom Apartment On 2 Floors + Bq Etc In Central Ikoyi'

In [7]:
# listing type
property_info.find("div", class_= "similar-listings-info").find('h3').get_text()

'3 BEDROOM FLAT & APARTMENT For Rent'

In [8]:
# property price
property_info.find("div", class_= "similar-listings-price").find('h4').get_text()

'₦ 45,000,000'

In [9]:
# property price currency
property_info.find("div", class_= "similar-listings-price").find('span').get_text()

'₦'

In [10]:
# Property size sqm
property_info.find('div', class_='similar-listings-info').find('h2').find('a').get_text()

'Fully Furnished 3 Bedroom Apartment On 2 Floors + Bq Etc In Central Ikoyi'

In [11]:
# furnishing status
property_info.find('div', class_='similar-listings-info').find('a').get_text()

'Fully Furnished 3 Bedroom Apartment On 2 Floors + Bq Etc In Central Ikoyi'

In [12]:
# property location
property_info.find("div", class_= "similar-listings-info").find('p').get_text()

'\n\n\n Central Ikoyi Ikoyi Lagos'

In [13]:
# Property agent link
property_info.find('div', class_='similar-listing-contact').find('a').get('href')

'/estate-agents/princeademolaosinugainternational'

In [14]:
# property date
property_info.find("div", class_= "media-body").find('h5').get_text()

'Updated 23 Nov 2025, Added 23 Jul 2025'

In [15]:
# property amenity
def property_amenities(property_info):
    amenities = {"bedrooms": "N/A", "bathrooms": "N/A", "toilets": "N/A"}
    
    amenity_div = property_info.find("ul", class_="property-benefit")
    if not amenity_div:
        return amenities
    
    amenity_tags = amenity_div.find_all("li")
    
    # Use index positions
    if len(amenity_tags) >= 1:
        text = amenity_tags[0].get_text(strip=True)
        amenities["bedrooms"] = int(text) if text.isdigit() else "N/A"
    if len(amenity_tags) >= 2:
        text = amenity_tags[1].get_text(strip=True)
        amenities["bathrooms"] = int(text) if text.isdigit() else "N/A"
    if len(amenity_tags) >= 3:
        text = amenity_tags[2].get_text(strip=True)
        amenities["toilets"] = int(text) if text.isdigit() else "N/A"
    
    return amenities

In [16]:
property_amenities(property_info)

{'bedrooms': 3, 'bathrooms': 3, 'toilets': 4}

In [17]:
# property image url
def property_image_url(property_info):
   
    img_tag = property_info.find("img")
    if img_tag and img_tag.get("src"):
        return img_tag["src"]
    return None

In [18]:
property_image_url(property_info)

'data:image/png;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs='

In [19]:
# furnishing status is in the image url therefore extracting it from url

img_tag = property_info.find('img')
img_url = img_tag.get('data-src') or img_tag.get('data-original') or img_tag.get('src')

def furnishing_status(img_url: str) -> str:
    if not img_url:
        return "Not Specified"
    
    text = img_url.lower()
    
    if "semi-furnished" in text or "partially-furnished" in text or "partially furnished" in text:
        return "Semi-Furnished"
    elif "furnished" in text:
        return "Furnished"
    elif "unfurnished" in text:
        return "Unfurnished"
    else:
        return "Not Specified"


In [20]:
furnishing_status(img_url)

'Furnished'

# Property for Sale

In [21]:
# Importing Libraries

import requests
from bs4 import BeautifulSoup 
import re

# importing the library to scrap image
import os
from urllib.parse import urljoin

In [22]:
# Error handling function while loading the site and
# Defining the Header for using User-Agent approach to have access to the site info

URL ="https://privateproperty.ng/property-for-sale"

HEADER = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"
}

def make_connection_now(url):
    try:
        response = requests.get(url, headers=HEADER)
        if response.status_code == 200:
            soup_contents = BeautifulSoup(response.content, 'html.parser')
            return soup_contents
        else:
            print(f"Resources Not available! Status Code {response.status_code}")
    except Exception as e:
        print(f"An Error occurs. Message: {e}")

In [23]:
contents = make_connection_now(url = URL)

In [24]:
property_info_div = contents.find('div', class_="similar-listings-item")

In [25]:
# property title
property_info_div.find("div", class_= "similar-listings-info").find('h2').get_text()

'Luxury 5 Bedroom Detached Duplex With Elevator + Cinema + Pool'

In [26]:
# listing type
property_info_div.find("div", class_= "similar-listings-info").find('h3').get_text()

'5 BEDROOM DETACHED DUPLEX For Sale'

In [27]:
# property price
property_info_div.find("div", class_= "similar-listings-price").find('h4').get_text()

'₦ 1,700,000,000'

In [28]:
# property price currency
property_info_div.find("div", class_= "similar-listings-price").find('span').get_text()

'₦'

In [29]:
# Property size sqm
property_info_div.find('div', class_='similar-listings-info').find('h2').find('a').get_text()

'Luxury 5 Bedroom Detached Duplex With Elevator + Cinema + Pool'

In [30]:
# furnishing status
property_info_div.find('div', class_='similar-listings-info').find('a').get_text()

'Luxury 5 Bedroom Detached Duplex With Elevator + Cinema + Pool'

In [31]:
# property location
property_info_div.find("div", class_= "similar-listings-info").find('p').get_text()

'\n\n\n Diplomatic Zone Katampe Ext. Abuja Phase 2'

In [32]:
# Property agent link
property_info_div.find('div', class_='similar-listing-contact').find('a').get('href')

'/estate-agents/innerworkings'

In [33]:
# property date
property_info_div.find("div", class_= "media-body").find('h5').get_text()

'Updated 23 Nov 2025, Added 21 Oct 2025'

In [34]:
# property amenity
def property_amenities(property_info_div):
    amenities = {"bedrooms": "N/A", "bathrooms": "N/A", "toilets": "N/A"}
    
    amenity_div = property_info_div.find("ul", class_="property-benefit")
    if not amenity_div:
        return amenities
    
    amenity_tags = amenity_div.find_all("li")
    
    # Use index positions
    if len(amenity_tags) >= 1:
        text = amenity_tags[0].get_text(strip=True)
        amenities["bedrooms"] = int(text) if text.isdigit() else "N/A"
    if len(amenity_tags) >= 2:
        text = amenity_tags[1].get_text(strip=True)
        amenities["bathrooms"] = int(text) if text.isdigit() else "N/A"
    if len(amenity_tags) >= 3:
        text = amenity_tags[2].get_text(strip=True)
        amenities["toilets"] = int(text) if text.isdigit() else "N/A"
    
    return amenities

In [35]:
property_amenities(property_info_div)

{'bedrooms': 5, 'bathrooms': 5, 'toilets': 6}

In [36]:
# property image url
def property_image_url(property_info_div):
    
    img_tag = property_info.find("img")
    if img_tag and img_tag.get("src"):
        return img_tag["src"]
    return None

In [37]:
property_image_url(property_info_div)

'data:image/png;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs='

In [38]:
# furnishing status is in the image url therefore extracting it from url
img_tag = property_info_div.find('img')
img_url = img_tag.get('data-src') or img_tag.get('data-original') or img_tag.get('src')

def furnishing_status_from_url(img_url: str) -> str:
    if not img_url:
        return "Not Specified"
    
    text = img_url.lower()
    
    if "semi-furnished" in text or "partially-furnished" in text:
        return "Partially Furnished"
    elif "furnished" in text:
        return "Furnished"
    elif "unfurnished" in text:
        return "Unfurnished"
    else:
        return "Not Specified"


In [39]:
furnishing_status_from_url(img_url)

'Not Specified'